## Índice de funciones utilizadas

In [ ]:
"""
1. Estadísiticos:
    
    Slope:
        FeatureSpectralSlope(input_signal, f_s)
        Input: 
            input_signal: señal
            f_s: frecuencia de muestreo
        Output: 
            slope
    
    
    Centroide:
        centroide_espectral(input_signal)
        Input:
            input_signal
        Output:
            centroide

    
    Tabla con estadísticos
        muestraestadisticos(input_signal,fs, namesignal)
        Input:
            input_signal
            fs: Frecuencia de muestreo
            namesignal: nombre de la señal
        Output:
            Dataframe con los estadísticos del canal a analizar.

        
    Muestra kurtosis:
        muestrakurtosis(input_signal, namesignal)
        Input:
            input_signal
            namesignal
        Output:
            Mensaje con la kurtosis y nombre de canal a analizar.

        
2. Filtros

    Filtro de paso alto/bajo:
        highlowfilter(type, input_signal)
        Input:
            type: 'low' si el filtro es de paso bajo, 'hp' si el filtro es de paso alto
            input_signal
        Output:
            y: señal filtrada

    
    Cepstrum:
        filtradoCepstrum(input_signal, t, fs)
        Input:
            input_signal
            t
            fs
        Output:
            filtradatotal: señal filtrada (complejo)
            filtradareal: parte real de la señal filtrada.

        
    Gráfica señal filtrada y original
        muestrafiltro(input_signal, filtrada, t, opacidad)
        Input:
            input_signal
            filtrada: señal filtrada
            t: Tiempo
            y: señal filtrada obtenida en highlowfilter()
            opacidad: nivel alpha para gráfica
        Output:
            Gráfica de las dos señales


3. Kurtogramas

    Fast Kurtogram
        Fast_Kurtogram(x, nlevel, verbose=False, Fs=1, NFIR=16, fcut=0.4, opt1=1, opt2=1)
        Input:
            x: señal a analizar
            nlevel: número de niveles de descomposición del kurtograma
            Fs= fs        
        Output:
            Kwav: kurtograma
            Level_w: Vector de niveles
            freq_w: vector de frecuencias.
            c: envelope complejo de la señal filtrada en la banda de frecuencia de máxima kurtosis
            f_lower: 
            f_upper: 
        
    Ancho de banda y centro de frecuencia donde se encuentra la kurtosis máxima
        getBandwidthAndFrequency(nlevel, Fs, level_w, freq_w, level_index, freq_index)
        Input:
            nlevel
            Fs
            level_w
            freq_w
            level_index: 
            freq_ index:       
        Output:
            bw: Ancho de banda
            fc: Centro del intervalo de frecuencias
            fi: 
            l1: nivel
    
    Muestra kurtograma
        plotKurtogram(Kwav, level_w, f, freq_w)
        Input:
            Kwav
            level_w
            f:
            freq_w
        Output: 
            Gráfica con el kurtograma.


4. Envelope spectrum

    Muestra gráfica envelope spectrum con armónicos
        envelope_spectrum(signal, fs, BPFI, BPFO, title)
        Input:
            signal:
            fs:
            BPFI:
            BPFO:
            title:
        Output:
            fSpec: vector de frecuencias
            xSpec: Picos 
            fSpecG: Frecuencias a las que se encuentran dichos picos
            BPFI_coords: Vector con los armónicos BPFI 
            BPFO_coords: Vector con los armónicos BPFO
            
    
    Calcula el porcentaje de armónicos comunes de la señal con BPFI y BPFO
        por_comunes(fSpecG, BPFI_coords, BPFO_coords, fs, fr)
        Input:
            fSpecG:
            BPFI_coords:
            BPFO_coords:
            fr:
        Output:
            por_comunes_BPFI: Porcentaje de comunes de la señal con BPFI
            por_comuunes_BPFO: Porcentaje de comunes de la señal con BPFO
            
            
5. Descomposición en IMFs de la señal

    Descomposición utilizando VMD (Variational Mode Decomposition)
        descompvmd(input_signal, alpha, numIMFs)
        Input:
            input_signal
            alpha
            numIMFs: Número de IMFs en las que descomponer la señal
        Output:
            u: Conjunto de todas las señales 1ª IMF: u[0]. Última: u[numIMFs-1]
            filtradavmd: Señal filtrada. u[1]+...+u[numIMFs-1]
            
    Muestra la descomposición de la señal en sus distintas IMFs
        muestraimfvmd(input_signal, u)
        Input:
            input_signal:
            u
        Output:
            Gráfica con la señal original y las distintas IMFs
            
6. Gráfica de FFT y sus armónicos

        armonicosyfft(input_signal, t, BPFI_coords, BPFO_coords)
        Input: 
            input_signal:
            t:
            BPFI_coords:
            BPFO_coords:
        Output:
            Gráfica que muestra FFT utilizando la función ventana y los distintos armónicos.
        
            
            
        


"""

In [ ]:
logging.basicConfig(level=logging.INFO,
                    format='%(levelname)s : %(asctime)s : %(message)s')


eps = np.finfo(float).eps

## Paquetes necesarios

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, fftshift, ifft, ifftshift
from scipy import stats
from scipy.stats import skew, kurtosis
import cmath
import os
import glob
import logging
import scipy.signal as si
from obspy.core import utcdatetime
from scipy.signal import hilbert, chirp
from scipy.signal import find_peaks
from vmdpy import VMD 
import scipy.io as sio 



### Estadísticos
#### Se muestran a continuación las funciones para calcular algunos estadísticos

#### Spectral Slope

In [ ]:
def FeatureSpectralSlope(input_signal, f_s):   
    mu_x = np.mean(input_signal)    
    kmu = np.arange(0,len(input_signal)) - len(input_signal)/2   
    input_signal = input_signal - mu_x 
    vssl = np.dot(kmu, input_signal) / np.dot(kmu, kmu)

    return (vssl)

#### Centroid

In [ ]:
def centroide_espectral(input_signal):  
    espectro = abs(fft(input_signal))
    espectro_normalizado = espectro / sum(espectro)
    frec_normalizadas = np.linspace(0, 1, len(espectro))
    centroide_espectral = sum(espectro_normalizado * frec_normalizadas)
    
    return centroide_espectral

#### Muestra estadísticos: Realiza una tabla con los estadísticos del canal

In [ ]:
def muestraestadisticos(input_signal,fs, namesignal):
    #Estadísticos en el dominio de tiempo
    media = np.mean(input_signal)
    varianza = input_signal.var()    
    skewness = skew(input_signal)
    kurt = kurtosis(input_signal)
    
    #Momentos de orden 3 y 4 
    sumsignal = 0
    sum2 = 0
    sum3 = 0
    sum4 = 0    
    
    for i in range(0,len(input_signal)-1):
        sum2 = sum2 + (input_signal[i]**2)
        sum3 = sum3 + (input_signal[i]**3)
        sum4 = sum3 + (input_signal[i]**4)
       
    momento3 = ((sum3)/len(input_signal)) - 3*(media*(sum2/len(input_signal))) + 2*(media**3)
    momento4 = ((sum4)/len(input_signal)) - 4*(media*(sum3/len(input_signal))) + 6*((media**2)*(sum2/len(input_signal))) - 3*(media**4)
    
    centroide = centroide_espectral(input_signal)
    slope = FeatureSpectralSlope(input_signal, fs)
    
    #Crea un data frame con los estadísticos
    #Notacion exponencial
    pd.set_option('display.float_format', '{:e}'.format)

    stads = {'Estadísticos': ['Varianza', 'Momento de orden 3', 'Momento de orden 4', 'Skewness', 'Kurtosis',
                          'Centroid', 'Slope'],
           namesignal : [varianza, momento3, momento4, skewness, kurt, centroide, slope]      
           }

    estadisticos = pd.DataFrame(stads)
    return estadisticos

#### Muestra la kurtosis del canal

In [ ]:
def muestrakurtosis(input_signal, namesignal):
    kurt = kurtosis(input_signal)
    #Quiero que me muestre el nombre de la señal
    print('La kurtosis del canal', namesignal, 'es', kurt)

### Funciones

#### Filtro de paso alto/bajo

In [ ]:
def highlowfilter(type, input_signal):
    #type: 'hp' high pass, 'low': low pass    
    b, a = si.butter(3, 0.05,type) 
    zi = si.lfilter_zi(b, a)
    z, _ = si.lfilter(b, a, input_signal, zi=zi*input_signal[0])
    z2, _ = si.lfilter(b, a, z, zi=zi*z[0])
    y = si.filtfilt(b, a, input_signal)    
    
    return y

#### Filtra la señal utilizando Cepstrum

In [ ]:
def filtradoCepstrum(input_signal, t, fs):
    dt = t[1]-t[0]
    n = len(input_signal)
    fftsignal = fft(input_signal)
    frec = fftfreq(n,dt)
    
    #Calculamos la fase y la amplitud
    fase = np.unwrap(np.angle(fftsignal))
    amplitud = abs(fftsignal)
    
    #Manipulamos la amplitud:
    logampli = np.log(amplitud)
    #Obtenemos el ceptrum real:
    realceps = np.fft.ifftshift(logampli)
    
    #Realizamos el filtro de paso bajo
    edit_rcep = highlowfilter('low', realceps)
    #Realizamos la FFT y nos quedamos con la amplitud
    editampli = abs(np.fft.fftshift(edit_rcep))
    edit_ampli = np.log(editampli)
    
    #Unimos la fase y la amplitud
    unionedit = fase*1j + edit_ampli
    
    #Realizamos la exponencial de este número para obtener el complejo en forma exponencial
    cepscom = []
    for i in range(0,len(unionedit)):
        cepscom.append(cmath.exp(unionedit[i]))
    cepscom = np.array(cepscom)
    
    #Realizamos la funcion inversa y obtenemos la señal filtrada
    filtradatotal = np.fft.ifft(cepscom)
    filtradareal = filtradatotal.real

    return filtradatotal, filtradareal
    

In [ ]:
def muestrafiltro(input_signal, filtradareal, t, opacidad):
    
    fig = plt.figure(figsize=(30,10))
    plt.plot(t, input_signal, 'b', alpha=opacidad)
    plt.plot(t, filtradareal, 'g', alpha=opacidad)
    plt.title('Canal original y canal filtrado')
    
    plt.show()  

#### Kurtograma 

In [ ]:
def Fast_Kurtogram(x, nlevel, verbose=False, Fs=1, NFIR=16, fcut=0.4,
                   opt1=1, opt2=1):
    """
    Computes the fast kurtogram Kwav of signal x up to level 'nlevel'
    Maximum number of decomposition levels is log2(length(x)), but it is
    recommended to stay by a factor 1/8 below this.
    Also returns the vector of k-levels Level_w, the frequency vector 
    freq_w, the complex envelope of the signal c and the extreme 
    frequencies of the "best" bandpass f_lower and f_upper.

    J. Antoni : 02/2005
    Translation to Python: T. Lecocq 02/2012

    :param x: signal to analyse
    :param nlevel: number of decomposition levels
    :param verbose: If ``True`` outputs debugging information
    :param Fs: Sampling frequency of signal x
    :param NFIR: Length of FIR filter
    :param fcut: Fraction of Nyquist for filter
    :param opt1: [1 | 2]:
        * opt1 = 1: classical kurtosis based on 4th order statistics
        * opt1 = 2: robust kurtosis based on 2nd order statistics of the
        envelope (if there is any difference in the kurtogram between the
        two measures, this is due to the presence of impulsive additive
        noise)
    :param opt2: [1 | 2]:
        * opt2=1: the kurtogram is computed via a fast decimated filterbank
        * opt2=2: the kurtogram is computed via the short-time Fourier
        transform (option 1 is faster and has more flexibility than option
        2 in the design of the analysis filter: a short filter in option 1
        gives virtually the same results as option 2)

    :type x: numpy array
    :type nlevel: integer
    :type Fs: integer
    :type NFIR: integer
    :type fcut: float

    :returns: Kwav, Level_w, freq_w, c, f_lower, f_upper
        * Kwav: kurtogram
        * Level_w: vector of levels
        * freq_w: frequency vector
        * c: complex envelope of the signal filtered in the frequency band that maximizes the kurtogram
        * f_lower: lower frequency of the band pass
        * f_upper: upper frequency of the band pass
    """

    N = len(x)
    N2 = np.log2(N) - 7
    if nlevel > N2:
        logging.error('Please enter a smaller number of decomposition levels')

    # Fast computation of the kurtogram
    ####################################

    if opt1 == 1:
        # 1) Filterbank-based kurtogram
        ############################
        # Analytic generating filters

        h, g, h1, h2, h3 = get_h_parameters(NFIR, fcut)

        if opt2 == 1: # kurtosis of the complex envelope
            Kwav = K_wpQ(x, h, g, h1, h2, h3, nlevel, verbose, 'kurt2')
        else: # variance of the envelope magnitude
            Kwav = K_wpQ(x, h, g, h1, h2, h3, nlevel, verbose, 'kurt1')

        # keep positive values only!
        Kwav[Kwav <= 0] = 0
        Level_w = np.arange(1, nlevel+1)
        Level_w = np.array([Level_w, Level_w + np.log2(3.)-1])
        Level_w = sorted(Level_w.ravel())
        Level_w = np.append(0, Level_w[0:2*nlevel-1])
        freq_w = Fs*(np.arange(0, 3*2.0**nlevel)/(3*2**(nlevel+1)) +
                     1.0/(3.*2.**(2+nlevel)))

        M, index = get_GridMax(Kwav)
        level_index = index[0]
        freq_index = index[1]
        bw, fc, fi, l1 = getBandwidthAndFrequency(nlevel, Fs, Level_w, freq_w,
                                                  level_index, freq_index)

        if verbose:
            plotKurtogram(Kwav, freq_w, nlevel, Level_w, Fs, fi, level_index)

    else:
        logging.error('stft-based is not implemented')

    # Signal filtering !
    c = []
    test = 1
    lev = l1
    while test == 1:
        test = 0
        c, s, threshold, Bw, fc = Find_wav_kurt(x, h, g, h1, h2, h3,
                                                nlevel, lev, fi, Fs=Fs,
                                                verbose=verbose)

    # Determine the lowest and the uppest frequencies of the bandpass
    f_lower = Fs*np.round((fc-Bw/2.)*10**3)/10**3
    f_upper = Fs*np.round((fc+Bw/2.)*10**3)/10**3

    return Kwav, Level_w, freq_w, c, f_lower, f_upper


In [ ]:
def get_h_parameters(NFIR, fcut):
    """
    Calculates h-parameters used in Antoni (2005)

    :param NFIR: length of FIR filter
    :param fcut: fraction of Nyquist for filter

    :type NFIR: integer
    :type fcut: float

    :rtype: numpy array
    :returns: h-parameters: h, g, h1, h2, h3

    """

    h = si.firwin(NFIR+1, fcut) * np.exp(2*1j*np.pi*np.arange(NFIR+1) * 0.125)
    n = np.arange(2, NFIR+2)
    g = h[(1-n) % NFIR]*1/((-1)**(-(1-n)))
    NFIR = int(np.fix((3./2.*NFIR)))
    h1 = si.firwin(NFIR+1, 2./3*fcut)*np.exp(2j*np.pi*np.arange(NFIR+1) *
                                             0.25/3.)
    h2 = h1*np.exp(2j*np.pi*np.arange(NFIR+1)/6.)
    h3 = h1*np.exp(2j*np.pi*np.arange(NFIR+1)/3.)
    return (h, g, h1, h2, h3)

In [ ]:
def getBandwidthAndFrequency(nlevel, Fs, level_w, freq_w, level_index,
                             freq_index):
    """
    Gets bandwidth bw and frequency parameters knowing the 
    level and the frequency indexes.

    :param nlevel: number of decomposition levels
    :param Fs: sampling frequency of the signal
    :param level_w: vector of decomposition levels
    :param freq_w: vector of frequencies
    :param level_index: index of the level 
    :param freq_index: index of the frequency

    :type nlevel: integer
    :type Fs: integer
    :type level_w: numpy array
    :type freq_w: numpy array
    :type level_index: integer
    :type freq_index: integer

    :returns: bw, fc, fi, l1
        * bw: bandwidth
        * fc: central frequency
        * fi: index of the frequency sequence within the level l1
        * l1: level
    """

    #f1 = freq_w[freq_index]
    l1 = level_w[level_index]
    fi = (freq_index)/3./2**(nlevel+1)
    fi += 2.**(-2-l1)
    bw = Fs*2**-(l1)/2
    fc = Fs * fi

    return bw, fc, fi, l1


In [ ]:
def K_wpQ(x, h, g, h1, h2, h3, nlevel, verbose, opt, level=0):
    """
    Calculates the kurtosis K (2-D matrix) of the complete quinte wavelet packet 
    transform w of signal x, up to nlevel, using the lowpass and highpass filters 
    h and g, respectively. The WP coefficients are sorted according to the frequency
    decomposition. This version handles both real and analytical filters, but
    does not yield WP coefficients suitable for signal synthesis.

    J. Antoni : 12/2004
    Translation to Python: T. Lecocq 02/2012

    :param x: signal
    :param h: lowpass filter
    :param g: higpass filter
    :param h1: filter parameter returned by get_h_parameters
    :param h2: filter parameter returned by get_h_parameters
    :param h3: filter parameter returned by get_h_parameters
    :param nlevel: number of decomposition levels
    :param verbose: If ``True`` outputs debugging information
    :param opt: ['kurt1' | 'kurt2']
        * 'kurt1' = variance of the envelope magnitude
        * 'kurt2' = kurtosis of the complex envelope
    :param level: decomposition level for this call

    :type x: numpy array
    :type h: numpy array
    :type g: numpy array
    :type h1: numpy array
    :type h2: numpy array
    :type h3: numpy array
    :type nlevel: integer
    :type opt: string

    :returns: kurtosis

    """

    L = np.floor(np.log2(len(x)))
    if level == 0:
        if nlevel >= L:
            logging.error('nlevel must be smaller')
        level = nlevel
    x = x.ravel()
    KD, KQ = K_wpQ_local(x, h, g, h1, h2, h3, nlevel, verbose, opt, level)
    K = np.zeros((2*nlevel, 3*2**nlevel))

    K[0, :] = KD[0, :]
    for i in range(1, nlevel):
        K[2*i-1, :] = KD[i, :]
        K[2*i, :] = KQ[i-1, :]

    K[2*nlevel-1, :] = KD[nlevel, :]
    return K


In [ ]:
def K_wpQ_local(x, h, g, h1, h2, h3, nlevel, verbose, opt, level):
    """
    Is a recursive funtion. 
    Computes and returns the 2-D vector K, which contains the kurtosis value of the signal as 
    well as the 2 kurtosis values corresponding to the signal filtered into 2 different 
    band-passes.
    Also returns and computes the 2-D vector KQ which contains the 3 kurtosis values corresponding 
    to the signal filtered into 3 different band-passes.

    :param x: signal
    :param h: lowpass filter
    :param g: highpass filter
    :param h1: filter parameter returned by get_h_parameters
    :param h2: filter parameter returned by get_h_parameters
    :param h3: filter parameter returned by get_h_parameters
    :param nlevel: number of decomposition levels
    :param verbose: If ``True`` outputs debugging information
    :param opt: ['kurt1' | 'kurt2']
        * 'kurt1' = variance of the envelope magnitude
        * 'kurt2' = kurtosis of the complex envelope
    :param level: decomposition level for this call

    :type x: numpy array
    :type h: numpy array
    :type g: numpy array
    :type h1: numpy array
    :type h2: numpy array
    :type h3: numpy array
    :type nlevel: integer
    :type opt: string
    :type level: integer

    :returns: K, KQ

    """

    a, d = DBFB(x, h, g)

    N = len(a)
    d = d*np.power(-1., np.arange(1, N+1)) # indices pairs multipliés par -1
    K1 = kurt(a[len(h)-1:], opt)
    K2 = kurt(d[len(g)-1:], opt)

    if level > 2:
        a1, a2, a3 = TBFB(a, h1, h2, h3)
        d1, d2, d3 = TBFB(d, h1, h2, h3)
        Ka1 = kurt(a1[len(h)-1:], opt)
        Ka2 = kurt(a2[len(h)-1:], opt)
        Ka3 = kurt(a3[len(h)-1:], opt)
        Kd1 = kurt(d1[len(h)-1:], opt)
        Kd2 = kurt(d2[len(h)-1:], opt)
        Kd3 = kurt(d3[len(h)-1:], opt)
    else:
        Ka1 = 0
        Ka2 = 0
        Ka3 = 0
        Kd1 = 0
        Kd2 = 0
        Kd3 = 0

    if level == 1:
        K = np.array([K1*np.ones(3), K2*np.ones(3)]).flatten()
        KQ = np.array([Ka1, Ka2, Ka3, Kd1, Kd2, Kd3])
    if level > 1:
        Ka, KaQ = K_wpQ_local(a, h, g, h1, h2, h3, nlevel, verbose, opt,
                              level-1)

        Kd, KdQ = K_wpQ_local(d, h, g, h1, h2, h3, nlevel, verbose, opt,
                              level-1)

        K1 = K1*np.ones(np.max(Ka.shape))
        K2 = K2*np.ones(np.max(Kd.shape))
        K12 = np.append(K1, K2)
        Kad = np.hstack((Ka, Kd))
        K = np.vstack((K12, Kad))

        Long = int(2./6*np.max(KaQ.shape))
        Ka1 = Ka1*np.ones(Long)
        Ka2 = Ka2*np.ones(Long)
        Ka3 = Ka3*np.ones(Long)
        Kd1 = Kd1*np.ones(Long)
        Kd2 = Kd2*np.ones(Long)
        Kd3 = Kd3*np.ones(Long)
        tmp = np.hstack((KaQ, KdQ))

        KQ = np.concatenate((Ka1, Ka2, Ka3, Kd1, Kd2, Kd3))
        KQ = np.vstack((KQ, tmp))

    if level == nlevel:
        K1 = kurt(x, opt)
        K = np.vstack((K1*np.ones(np.max(K.shape)), K))

        a1, a2, a3 = TBFB(x, h1, h2, h3)
        Ka1 = kurt(a1[len(h)-1:], opt)
        Ka2 = kurt(a2[len(h)-1:], opt)
        Ka3 = kurt(a3[len(h)-1:], opt)
        Long = int(1./3*np.max(KQ.shape))
        Ka1 = Ka1*np.ones(Long)
        Ka2 = Ka2*np.ones(Long)
        Ka3 = Ka3*np.ones(Long)
        tmp = np.array(KQ[0:-2])

        KQ = np.concatenate((Ka1, Ka2, Ka3))
        KQ = np.vstack((KQ, tmp))

    return K, KQ


In [ ]:
def DBFB(x, h, g):
    """
    Double-band filter-bank.
    [a,d] = DBFB(x,h,g) computes the approximation
    coefficients vector a and detail coefficients vector d,
    obtained by passing signal x though a two-band analysis filter-bank.

    :param x: signal
    :param h: The decomposition low-pass filter and
    :param g: The decomposition high-pass filter.

    :type x: numpy array
    :type h: numpy array
    :type g: numpy array

    :rtype: numpy array
    :returns: a, d

    """

    # lowpass filter
    a = si.lfilter(h, 1, x)
    a = a[1::2]
    a = a.ravel()

    # highpass filter
    d = si.lfilter(g, 1, x)
    d = d[1::2]
    d = d.ravel()

    return (a, d)

In [ ]:
def kurt(x, opt):
    """
    Calculates kurtosis of a signal according to the option chosen

    :param x: signal
    :param opt: ['kurt1' | 'kurt2']
        * 'kurt1' = variance of the envelope magnitude
        * 'kurt2' = kurtosis of the complex envelope

    :type x: numpy array
    :type opt: string

    :rtype: float
    :returns: Kurtosis
    """
    if opt == 'kurt2':
        if np.all(x == 0):
            K = 0
            E = 0
            return K
        x = x - np.mean(x)
        E = np.mean(np.abs(x)**2)
        if E < eps:
            K = 0
            return K

        K = np.mean(np.abs(x)**4)/E**2

        if np.all(np.isreal(x)):
            K = K - 3
        else:
            K = K - 2

    if opt == 'kurt1':
        if np.all(x == 0):
            K = 0
            E = 0
            return K
        x = x - np.mean(x)
        E = np.mean(np.abs(x))
        if E < eps:
            K = 0
            return K

        K = np.mean(np.abs(x)**2)/E**2

        if np.all(np.isreal(x)):
            K = K-1.57
        else:
            K = K-1.27

    return K


In [ ]:
def TBFB(x, h1, h2, h3):
    """
    Triple-band filter-bank.
    [a1,a2,a3] = TBFB(x,h1,h2,h3)

    :param x: signal
    :param h1: filter parameter
    :param h2: filter parameter
    :param h3: filter parameter

    :type x: numpy array
    :type h1: numpy array
    :type h2: numpy array
    :type h3: numpy array

    :rtype: numpy array
    :returns: a1, a2, a3

    """

    # lowpass filter
    a1 = si.lfilter(h1, 1, x)
    a1 = a1[2::3]
    a1 = a1.ravel()

    # passband filter
    a2 = si.lfilter(h2, 1, x)
    a2 = a2[2::3]
    a2 = a2.ravel()

    # highpass filter
    a3 = si.lfilter(h3, 1, x)
    a3 = a3[2::3]
    a3 = a3.ravel()

    return (a1, a2, a3)

In [ ]:
def get_GridMax(grid):
    """
    Gets maximum of a nD grid and its unraveled index

    :param grid: an nD-grid
    :type param: numpy array

    :returns:
    * M : grid maximum
    * index : index of maximum in unraveled grid
    """

    index = np.argmax(grid)
    M = np.amax(grid)
    index = np.unravel_index(index, grid.shape)

    return M, index

In [ ]:
def Find_wav_kurt(x, h, g, h1, h2, h3, nlevel, Sc, Fr, Fs=1, verbose=False):
    """
    TODO flesh out this doc-string

    J. Antoni : 12/2004
    Translation to Python: T. Lecocq 02/2012

    :param x: signal
    :param h: lowpass filter
    :param g: highpass filter
    :param h1: filter parameter returned by get_h_parameters
    :param h2: filter parameter returned by get_h_parameters
    :param h3: filter parameter returned by get_h_parameters
    :param nlevel: number of decomposition levels
    :param Sc: Sc = -log2(Bw)-1 with Bw the bandwidth of the filter
    :param Fr: in the range [0, 0.5]
    :param Fs: Sampling frequency of signal x
    :param verbose: If ``True`` outputs debugging information

    :type x: numpy array
    :type h: numpy array
    :type g: numpy array
    :type h1: numpy array
    :type h2: numpy array
    :type h3: numpy array
    :type nlevel: integer
    :type Fr: float
    :type: Fs: integer

    :returns: c, s, threshold, Bw, fc
    """
    level = np.fix((Sc))+((Sc % 1) >= 0.5)*(np.log2(3)-1)
    Bw = 2**(-level-1)
    freq_w = np.arange(0, 2**level) / 2**(level+1) + Bw/2.
    J = np.argmin(np.abs(freq_w-Fr))
    fc = freq_w[J]
    i = int(np.round(fc/Bw-1./2))
    if level % 1 == 0:
        acoeff = binary(i, int(level))
        bcoeff = []
        temp_level = level
    else:
        i2 = int(np.fix((i/3.)))
        temp_level = np.fix((level))-1
        acoeff = binary(i2, int(temp_level))
        bcoeff = i-i2*3
    acoeff = acoeff[::-1]
    c = K_wpQ_filt(x, h, g, h1, h2, h3, acoeff, bcoeff, temp_level)

    t = np.arange(len(x))/float(Fs)
    tc = np.linspace(t[0], t[-1], len(c))
    s = np.real(c*np.exp(2j*np.pi*fc*Fs*tc))

    sig = np.median(np.abs(c))/np.sqrt(np.pi/2.)
    threshold = sig*raylinv(np.array([.999, ]), np.array([1, ]))

    return c, s, threshold, Bw, fc

In [ ]:
def binary(i, k):
    """
    Computes the coefficients of the binary expansion of i:
    i = a(1)*2^(k-1) + a(2)*2^(k-2) + ... + a(k)

    :param i: integer to expand
    :param k: nummber of coefficients

    :returns: coefficients a
    """

    if i >= 2**k:
        logging.error('i must be such that i < 2^k !!')

    a = np.zeros(k)
    temp = i
    for l in np.arange(k-1, -1, -1):
        a[k-l-1] = int(np.fix(temp/2**l))
        temp = temp - int(np.fix(a[k-l-1]*2**l))

    return a

In [ ]:
def K_wpQ_filt(x, h, g, h1, h2, h3, acoeff, bcoeff, level=0):
    """
    Calculates the kurtosis K of the complete quinte wavelet packet transform w
    of signal x, up to nlevel, using the lowpass and highpass filters h and g,
    respectively. The WP coefficients are sorted according to the frequency
    decomposition.
    This version handles both real and analytical filters, but does not yield
    WP coefficients suitable for signal synthesis.

    J. Antoni : 12/2004
    Translation to Python: T. Lecocq 02/2012

    :param x: signal
    :param h: lowpass filter
    :param g: highpass filter
    :param h1: filter parameter returned by get_h_parameters
    :param h2: filter parameter returned by get_h_parameters
    :param h3: filter parameter returned by get_h_parameters
    :param acoeff:
    :param acoeff:
    :param level:

    """

    nlevel = len(acoeff)
    L = np.floor(np.log2(len(x)))
    if level == 0:
        if nlevel >= L:
            logging.error('nlevel must be smaller !!')
        level = nlevel
    x = x.ravel()
    if nlevel == 0:
        if bcoeff == []:
            c = x
        else:
            c1, c2, c3 = TBFB(x, h1, h2, h3)
            if bcoeff == 0:
                c = c1[len(h1)-1:]
            elif bcoeff == 1:
                c = c2[len(h2)-1:]
            elif bcoeff == 2:
                c = c3[len(h3)-1:]
    else:
        c = K_wpQ_filt_local(x, h, g, h1, h2, h3, acoeff, bcoeff, level)
    return c

In [ ]:
def K_wpQ_filt_local(x, h, g, h1, h2, h3, acoeff, bcoeff, level):
    """
    Performs one analysis level into the analysis tree
    TODO : flesh out this doc-string

    :param x: signal
    :param h: lowpass filter
    :param g: higpass filter
    :param h1: filter parameter returned by get_h_parameters
    :param h2: filter parameter returned by get_h_parameters
    :param h3: filter parameter returned by get_h_parameters
    :param acoeff:
    :param acoeff:
    :param level:

    """

    a, d = DBFB(x, h, g)
    N = len(a)
    d = d*np.power(-1., np.arange(1, N+1))
    level = int(level)
    if level == 1:
        if bcoeff == []:
            if acoeff[level-1] == 0:
                c = a[len(h)-1:]
            else:
                c = d[len(g)-1:]
        else:
            if acoeff[level-1] == 0:
                c1, c2, c3 = TBFB(a, h1, h2, h3)
            else:
                c1, c2, c3 = TBFB(d, h1, h2, h3)
            if bcoeff == 0:
                c = c1[len(h1)-1:]
            elif bcoeff == 1:
                c = c2[len(h2)-1:]
            elif bcoeff == 2:
                c = c3[len(h3)-1:]
    if level > 1:
        if acoeff[level-1] == 0:
            c = K_wpQ_filt_local(a, h, g, h1, h2, h3, acoeff, bcoeff, level-1)
        else:
            c = K_wpQ_filt_local(d, h, g, h1, h2, h3, acoeff, bcoeff, level-1)

    return c

In [ ]:
def raylinv(p, b):
    """
    Inverse of the Rayleigh cumulative distribution function (cdf).
    X = RAYLINV(P,B) returns the Rayleigh cumulative distribution function
    with parameter B at the probabilities in P.

    :param p: probabilities
    :param b:

    """

    # Initialize x to zero.
    x = np.zeros(len(p))
    # Return NaN if the arguments are outside their respective limits.
    k = np.where(((b <= 0) | (p < 0) | (p > 1)))[0]

    if len(k) != 0:
        tmp = np.NaN
        x[k1] = tmp(len(k))

    # Put in the correct values when P is 1.
    k = np.where(p == 1)[0]
    if len(k) != 0:
        tmp = np.Inf
        x[k] = tmp(len(k))

    k = np.where(((b > 0) & (p > 0) & (p < 1)))[0]

    if len(k) != 0:
        pk = p[k]
        bk = b[k]
        x[k] = np.sqrt((-2*bk ** 2) * np.log(1 - pk))

    return x

#### Representación kurtograma a partir de las funciones anteriores

In [ ]:
def kurtogram_data(Kwav, level_w, f):
    Kmax = 0
    for i in range(0, len(Kwav) - 1):
        for j in range(0, len(Kwav[i])):
            if Kwav[i][j] > Kmax:
                Kmax = Kwav[i][j]
                ind_i = i
                ind_j = j

                
    len_win = 2**(1+level_w[ind_i])
    bw = f/len_win
    fc = ind_j/len(Kwav[ind_i])*f/2 + bw/2
    level_op = level_w[ind_i]
    
    return Kmax, level_op, len_win, fc, bw

In [ ]:
def plotKurtogram(Kwav, level_w, f, freq_w):
    Kmax, level_op, len_win, fc, bw  = kurtogram_data(Kwav, level_w, f)
    fig = plt.figure(figsize=(20,10))
    plt.imshow(Kwav, aspect='auto', extent=(0, freq_w[-1], range(2*nlevel)[-1],
                                            range(2*nlevel)[0]),
               interpolation='none', cmap=plt.cm.hot_r)
    xx = np.arange(0, int(freq_w[len(freq_w)-1]), step=5)
    plt.xticks(xx)
    plt.yticks(range(2*nlevel), np.round(level_w*10)/10)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Level k")
    plt.colorbar()
    
    plt.title("Kmax %.2f, Level %i, Optimal window length %i, \n Center frecuency=%.2f Hz, Bw=%.2f Hz" %
                  (Kmax, level_op, len_win, fc, bw))

### Envelope spectrum

In [ ]:
def envelope_spectrum(signal, fs, BPFI, BPFO, title):
    '''
    ENVELOPE SPECTRUM
    '''
    analytic_signal = hilbert(signal - np.mean(signal))
    amplitude_envelope = np.abs(analytic_signal)
    amplitude_envelope = amplitude_envelope - np.mean(amplitude_envelope)
    
    fSpec = []
    for i in range(0, len(amplitude_envelope)):
        fSpec.append(i/len(amplitude_envelope)*fs)
    xSpec = 1/len(amplitude_envelope)*abs(fft(amplitude_envelope));
    
    #Indice armónicos
    intervalo = []
    
    for i in range(0,len(fSpec)):
        if fSpec[i]<fr*2:
            intervalo.append(xSpec[i])
        
    
    intervalo = np.array(intervalo)
    intervalo2 = intervalo.tolist()
    BPF0 = max(intervalo)
    indice = intervalo2.index(max(intervalo2))
    
    
    
    # Compute one-sided spectrum. Compensate the amplitude for a two-sided
    # spectrum. Double all points except DC and nyquist.
    if len(xSpec) % 2 == 0: 
        # Even length two-sided spectrum
        fSpec = fSpec[0:int(len(fSpec)/2+1)]
        xSpec = xSpec[0:int(len(xSpec)/2+1)]
        xSpec[1:len(xSpec)-1] = 2*xSpec[1:len(xSpec)-1]

    else:
        # Odd length two-sided spectrum
        fSpec = fSpec[0:int((len(fSpec)+1)/2)]
        xSpec = xSpec[0:int((len(xSpec)+1)/2)]
        xSpec[1:len(xSpec)] = 2*xSpec[1:len(xSpec)]
        
    fig = plt.figure(figsize=(30,10))
    
    
    peaks_c, _ = find_peaks(xSpec, np.percentile(xSpec,99))
    peaks = xSpec[peaks_c]
    cota = np.mean(peaks)
    fSpecG = []
    for i in range(0,len(xSpec)):
        if xSpec[i] >= cota:
            fSpecG.append(fSpec[i])
    
    inicioarmon = fSpec[indice]
    plt.plot(fSpec,xSpec, label='Envelope spectrum')
    #BPFI_coords = np.arange(0, fSpecG[len(fSpecG) - 1] + fs/1000, BPFI)
    BPFI_coords = np.arange(inicioarmon, fs/2, BPFI)
    for xc in BPFI_coords:
        if xc == BPFI_coords[0]:
            plt.axvline(x=xc, color = 'r', linestyle = '--', lw=1.5, alpha = 0.5,label='BPFI')
        else:
            plt.axvline(x=xc, color = 'r', linestyle = '--', lw=1.5, alpha = 0.5)
    #BPFO_coords = np.arange(0, fSpecG[len(fSpecG) - 1] + fs/1000, BPFO)
    BPFO_coords = np.arange(inicioarmon, fs/2, BPFO)
    for xc2 in BPFO_coords:
        if xc2 == BPFO_coords[0]:
            plt.axvline(x=xc2, color = 'g', linestyle = '--', lw=1.5, alpha = 0.5, label='BPFO')
        else:
            plt.axvline(x=xc2, color = 'g', linestyle = '--', lw=1.5, alpha = 0.5)
            
    plt.title(title, fontsize=25)
    plt.legend(fontsize=20)
    
    return fSpec, xSpec, fSpecG, BPFI_coords, BPFO_coords

#### Muestra el porcentaje de coincidencias de armónicos

In [ ]:
def por_comunes(fSpecG, BPFI_coords, BPFO_coords, fs, fr):
    '''
    Porcentaje de coincidencias entre el envelope spectrum y 
    las rectas correspondientes con BPFI y BPFO
    '''
    BPFI_array_v = BPFI_coords
    comunes_BPFI = 0
    for f in fSpecG:
        for i in range(0,len(BPFI_array_v)-1):
            if f < BPFI_array_v[i] + fr and f > BPFI_array_v[i] - fr:
                comunes_BPFI = comunes_BPFI + 1
                BPFI_array_v = np.delete(BPFI_array_v, i)
                break
    
    por_comunes_BPFI = comunes_BPFI * 100 / min(len(BPFI_coords), len(fSpecG))
                                               
    BPFO_array_v = BPFO_coords
    comunes_BPFO = 0
    for f in fSpecG:
        for i in range(0,len(BPFO_array_v)-1):
            if f < BPFO_array_v[i] + fr and f > BPFO_array_v[i] - fr:
                comunes_BPFO = comunes_BPFO + 1
                BPFO_array_v = np.delete(BPFO_array_v, i)
                break

    por_comunes_BPFO = comunes_BPFO * 100 / min(len(BPFO_coords), len(fSpecG))
    
    return por_comunes_BPFI, por_comunes_BPFO
    

### Descomposición IMF de la señal

#### VMD

In [ ]:
def descompvmd(input_signal, alpha, numIMFs ):
    tau = 0
    DC = 0
    init = 1
    tolerancia = 1e-7
    #alpha = 5000
    filtradavmd = 0
    u, u_hat, omega = VMD(input_signal, alpha, tau, numIMFs, DC, init, tolerancia)
    for i in range(1,numIMFs):
        #u[0] contiene la señal con ruido. La señal a analizar es la suma del resto de las IMFs
        filtradavmd = filtradavmd + u[i]
    
    return filtradavmd, u 



In [ ]:
def muestraimfvmd(input_signal, u):
    
    plt.figure()
    plt.subplot(2,1,1)
    plt.plot(input_signal)
    plt.title('Señal original')
    plt.xlabel('tiempo (s)')
    plt.subplot(2,1,2)
    plt.plot(u.T)
    plt.title('Descomposición')
    plt.xlabel('Tiempo (s)')
    plt.legend(['Mode %d'%m_i for m_i in range(u.shape[0])])
    plt.tight_layout()

### Muestra armónicos utilizando la FFT


In [ ]:
def armonicosyfft(input_signal, t, BPFI_coords, BPFO_coords):
    
    
    dt = t[1]-t[0]
    n = len(input_signal)
    fftsignal = fftshift(fft((input_signal- np.mean(input_signal))*np.hanning(n))/n)
    frec = fftfreq(n,dt)
    frec = np.fft.fftshift(frec)

    fig = plt.figure(figsize=(30,10))
    plt.plot(frec,abs(fftsignal.imag), label='FFT y armónicos')
    for xc in BPFI_coords:
        if xc == BPFI_coords[0]:
            plt.axvline(x=xc, color = 'r', linestyle = '--', lw=1.5, alpha = 0.5,label='BPFI')
        else:
            plt.axvline(x=xc, color = 'r', linestyle = '--', lw=1.5, alpha = 0.5)
    

    for xc2 in BPFO_coords:
        if xc2 == BPFO_coords[0]:
            plt.axvline(x=xc2, color = 'g', linestyle = '--', lw=1.5, alpha = 0.5, label='BPFO')
        else:
            plt.axvline(x=xc2, color = 'g', linestyle = '--', lw=1.5, alpha = 0.5)
    plt.xlim(0,100000)